In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

from pdflight import in_task, shift_datetime

%matplotlib inline

In [ ]:
with open("store.pkl", "rb") as f:
    stored = pickle.load(f)
    md, fl, th = stored["md"], stored["fl"], stored["th"]
md.competition.unique()

In [ ]:
competition = md.index[
    md.competition == "osterreichische-segelflugstaats-und-bundesmeisterschaften-2025"
]
if True:
    md = md.loc[competition]
    fl = fl.loc[competition]
    th = th.loc[competition]
fl

In [ ]:
tt = th.loc[in_task(th, md) & (th.dt.total_seconds() >= 120)]

In [ ]:
df = tt.reset_index()
df["h_enter"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].alt.values
df["v_enter"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].gsp.values
df["e_enter"] = df.h_enter * 9.81 + df.v_enter**2 / 2
df["exit"] = df.datetime + df.duration
df["h_exit"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].alt.values
df["v_exit"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].gsp.values
df["e_exit"] = df.h_exit * 9.81 + df.v_exit**2 / 2
df["dh"] = df.h_exit - df.h_enter
df["vv"] = df.dh / df.duration.dt.total_seconds()
df["de"] = df.e_exit - df.e_enter
df["p"] = df.de / df.duration.dt.total_seconds()
if True:
    dt_stable = 40
    df["stable"] = df.datetime + pd.Timedelta(seconds=dt_stable)
    df["h_stable"] = fl.loc[
        pd.MultiIndex.from_frame(df[["flight", "stable"]])
    ].alt.values
    df["v_stable"] = fl.loc[
        pd.MultiIndex.from_frame(df[["flight", "stable"]])
    ].gsp.values
    df["e_stable"] = df.h_stable * 9.81 + df.v_stable**2 / 2
    df["dh_stable"] = df.h_stable - df.h_enter
    df["vv_stable"] = df.dh_stable / dt_stable
    df["de_stable"] = df.e_stable - df.e_enter
    df["p_stable"] = df.de_stable / dt_stable
    df["eta_vv_stable"] = df.vv_stable / df.vv
    df["eta_p_stable"] = df.p_stable / df.p
    df["loss_e_stable"] = (df.p_stable - df.p) * dt_stable
df

In [ ]:
cc = md.join(
    df.loc[df.loss_e_stable.abs() < 2000].groupby("flight").mean(), how="inner"
)
sns.histplot(cc.loss_e_stable.sort_values())

In [ ]:
x = "loss_e_stable"
y = "points"
sns.regplot(data=cc, x=x, y=y, line_kws={"color": "red"})
stats.linregress(cc[x], cc[y])

In [ ]:
sns.histplot(cc.v_exit)